# 5 模型与分词器

In [18]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

## 5.1 模型

除了像之前使用 `AutoModel` 根据 checkpoint 自动加载模型以外，我们也可以直接使用模型对应的 `Model` 类，例如 BERT 对应的就是 `BertModel`。

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

注意：大部分情况下，都应该使用 `AutoModel` 来加载模型。这样如果我们想要更换模型，只需要修改 checkpoint，其他代码可以保持不变。

### 加载模型

所有存储在 HuggingFace Model Hub 上的模型都可以通过 `Model.from_pretrained()` 来加载权重，参数可以像上面一样是 checkpoint 的名称，也可以是模型的本地路径（预先下载的模型目录），例如：

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("./models/bert/")

`Model.from_pretrained()` 会自动缓存下载的模型权重，默认缓存路径是 `~/.cache/huggingface/transformers`，可以通过设置环境变量 `HF_HOME` 来修改缓存路径：`os.environ["HF_HOME"] = "/path/to/cache"`。

### 保存模型

保存模型通过调用 `Model.save_pretrained()` 方法，可以保存模型的权重、配置和词表等信息。例如保存加载的 BERT 模型：

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")
model.save_pretrained("./models/bert-base-cased/")

这会在保存路径下创建两个文件：

- `config.json`：模型的配置信息，包括模型的参数、词表等。
- `pytorch_model.bin`：又称为 state dict，保存了模型的权重。

我们自己保存的模型同样可以通过 `Model.from_pretrained()` 来加载，只需要传递自己保存的路径即可。

## 5.2 分词器

由于神经网络不能直接处理文本，因此我们需要先将文本转换为数字，这个过程被成为编码（Encoding），其中包含两个步骤：

1. 分词（Tokenization）：将文本分割为单词、子词或字符等，这些单元被称为 token;
2. 数字化（Numericalization）：将 token 映射为数字，例如词表中的索引。

### 分词策略

根据切分的粒度不同，分词策略可以分为以下几种：

#### 按词切分（Word-based）

将文本切分为单词，例如：`"I love HuggingFace"` -> `["I", "love", "HuggingFace"]`。

例如直接利用 Python 的 `split()` 函数按空格进行分词：

In [ ]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)  # ['Jim', 'Henson', 'was', 'a', 'puppeteer']

这种策略的问题是会将文本中所有出现过的单词都切分出来，从而产生巨大的词表。而实际上很多单词都是相关的，例如 "dog" 和 "dogs"，"run" 和 "running"，如果给他们分配不同的索引，就无法表示他们之间的关联性。

> 词表就是一个映射字典，负责将 token 映射为数字，例如：`{"I": 0, "love": 1, "HuggingFace": 2}`。神经网络模型就是通过这些 token ID 来区分每一个 token 的。

在遇到不在词表中的单词时（OOV，Out-Of-Vocabulary），分词器会使用一个专门的 token 来表示它是 unknown 的，例如：`[UNK]`。显然，如果分词结果中包含很多 `[UNK]`，就意味着丢失了很多文本信息，因此一个好的分词策略应该尽量减少 `[UNK]` 的出现。

#### 字符切分（Character-based）

这种策略把文本切分为字符而不是词语，这样就只会产生一个非常小的词表，并且很少会出现词表外的 tokens。

例如：`"I love HuggingFace"` -> `["I", " ", "l", "o", "v", "e", " ", "H", "u", "g", "g", "i", "n", "g", "F", "a", "c", "e"]`。

但是从直觉上来看，字符本身并没有太大的意义，因此将文本切分为字符之后就会变得不容易理解。

#### 子词切分（Subword-based）

高频词直接保留，低频词被切分为更有意义的子词，这样既减少了词表的大小，又保留了词语的意义。例如：`"HuggingFace"` -> `["Hug", "##ging", "##Face"]`，其中 `##` 表示这是一个词的一部分。这种策略可以很好地平衡词表大小和语义信息。

#### 加载与保存分词器

分词器的加载与保存与模型类似，也是通过 `Tokenizer.from_pretrained()` 和 `Tokenizer.save_pretrained()` 来实现的。例如加载并保存 BERT 分词器：

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer.save_pretrained("./models/bert-base-cased/")

同样的，在大部分情况下我们应该使用 `AutoTokenizer` 来加载分词器，这样可以保证代码的通用性。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer.save_pretrained("./models/bert-base-cased")

调用 `Tokenizer.save_pretrained()` 函数会在保存路径下创建三个文件：

- `tokenizer_config.json`：分词器的配置信息，包括分词器的参数、词表等；
- `special_tokens_map.json`：特殊 token 的映射信息，例如 `[CLS]`、`[SEP]` 和 `[UNK]` 等特殊字符的映射关系；
- `vocab.txt`：词表文件，一行一个 token，行号就是对应的 token ID；

#### 编码与解码文本

前面提到，分词器的主要功能是将文本转换为 token ID，这个过程被称为编码（Encoding），其中包含两个步骤：

1. 分词：使用分词器按照某种策略将文本切分为 tokens；
2. 映射：将 tokens 映射为 token IDs；

下面首先使用 BERT 分词器对文本进行分词：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple."
tokens = tokenizer.tokenize(sequence)

print(tokens)  # ['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple', '.']


['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple', '.']


可以看到，BERT 分词器采用的是子词切分的策略，例如 "transformer" 被切分为了 "transform" 和 "##er"。

然后，我们通过 `convert_tokens_to_ids()` 方法将 tokens 映射为 token IDs：

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)  # [7993, 170, 13809, 23763, 2443, 1110, 3014, 119]

[7993, 170, 13809, 23763, 2443, 1110, 3014, 119]


此外，还可以通过 `encode()` 函数将这两个步骤合并到一起，并且 `encode()` 会自动添加特殊 token，例如 BERT 分词器会分别在序列首尾添加 `[CLS]` 和 `[SEP]`：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple."
sequence_ids = tokenizer.encode(sequence)

print(sequence_ids)  # [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 102]

[101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 102]


其中 `101` 和 `102` 分别是 `[CLS]` 和 `[SEP]` 的 token ID。

注意：上面只是为了演示，在实际进行文本编码时，最常见的用法是直接使用分词器进行处理，这样不仅会返回 token IDs，还会返回 attention mask、token type IDs 等信息。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_text = tokenizer("Using a Transformer network is simple.")
print(tokenized_text)
"""
{
    'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 102],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}
"""

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


文本解码（Decoding）是编码的逆过程，即将 token IDs 转换为文本。注意：解码过程不是简单地将 token IDs 映射回 tokens，还需要合并那些被分为多个 token 的单词。下面通过 `decode()` 函数解码前面生成的 token IDs:


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

decoded_string = tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 3014, 119])
print(decoded_string)
decoded_string = tokenizer.decode([101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 119, 102])
print(decoded_string)

Using a Transformer network is simple.
[CLS] Using a Transformer network is simple. [SEP]


## 5.3 处理多段文本

现实场景中，我们往往会同时处理多段文本，而且模型也只接受批（batch）数据作为输入，即使只有一段文本，也需要将它组成一个只包含一个样本的 batch，例如：

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print("Inputs Keys:\n", tokenized_inputs.keys())
# dict_keys(['input_ids', 'attention_mask'])
print("\nInput IDs:\n", tokenized_inputs["input_ids"])
# tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
#           2607,  2026,  2878,  2166,  1012,   102]])
output = model(**tokenized_inputs)
print("\nLogits:\n", output.logits)
# tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>)

#### Padding 与 Attention Mask

按批输入多短文本产生的一个直接问题就是 batch 中的文本有长有短，而输入张量必须是严格的二维矩形，维度为 `[batch_size, max_length]`，因此需要对短文本进行 padding 操作，即在短文本后面填充特殊 token，使得所有文本的长度都相同。

模型的 padding token ID 可以通过其分词器的 `pad_token_id` 属性获取。

同时，我们在进行 Padding 操作时，必须明确告知模型哪些 token 是 padding token，这些 padding token 不应该参与模型的计算，这就是 Attention Mask 的作用。**Attention Mask 是一个与输入张量维度相同的张量，其中 padding token 对应的位置为 0，其他位置为 1**。

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]
batched_attention_masks = [
    [1, 1, 1],
    [1, 1, 0],
]

print(model(torch.tensor(sequence1_ids)).logits)  # tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
print(model(torch.tensor(sequence2_ids)).logits)  # tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(batched_attention_masks))
print(outputs.logits)
# tensor([[ 1.5694, -1.3895],
#         [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


在实际使用时，应该直接使用分词器对文本进行处理，它不仅会向 token 序列中添加模型需要的特殊字符（例如 `[CLS]`、`[SEP]`），还会自动生成对应的 Attention Mask 等信息。

目前大部分 Transformer 模型只能接受长度不超过 512 或 1024 的 token 序列，因此对于长序列，有以下三种处理方法：

1. 使用一个支持长文的 Transformer 模型，例如 Longformer 和 LED（最大长度 4096）；
2. 设定最大长度 `max_sequence_length` 以截断输入序列：`sequence = sequence[:max_sequence_length]`；
3. 将长文切片为短文本块（chunk），然后分别对每一个 chunk 编码。

#### 直接使用分词器

正如前面提到的，直接使用分词器对文本进行处理是最简单的方法，它会自动添加特殊 token，并生成 Attention Mask 等信息。例如：

In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

model_inputs = tokenizer(sequences)
print(model_inputs)
# {
#     'input_ids': [
#         [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102],
#         [101, 2061, 2031, 1045, 999, 102]
#     ],
#     'attention_mask': [
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#         [1, 1, 1, 1, 1, 1]
#     ]
# }

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


可以看到，分词器的输出包含了模型需要的所有输入项。对于 DistilBERT，就是 `input_ids` 和 `attention_mask`。

Padding 操作通过 padding 参数来控制：

- `padding="longest"`：将序列填充到当前 batch 中最长序列的长度；
- `padding="max_length"`：将序列填充到模型能够接受的最大长度，例如 BERT 模型就是 512；

In [9]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

model_inputs = tokenizer(sequences, padding="longest")
print(model_inputs)
# {
#     'input_ids': [
#         [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102],
#         [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
#     'attention_mask': [
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
#         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
# }


{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


**截断操作**通过 `truncation` 参数来控制，如果 `truncation=True`，那么大于模型最大接受长度的序列都会被截断，例如对 BERT 模型就会截断长度超过 512 的序列。此外，也可以通过 `max_length` 参数来指定截断的长度：

In [10]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

model_inputs = tokenizer(sequences, max_length=8, padding="longest", truncation=True)
print(model_inputs)
# {
#     'input_ids': [
#         [101, 1045, 1005, 2310, 2042, 3403, 2005, 102],
#         [101, 2061, 2031, 1045, 999, 102, 0, 0]],
#     'attention_mask': [
#         [1, 1, 1, 1, 1, 1, 1, 1],
#         [1, 1, 1, 1, 1, 1, 0, 0]]
# }

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0]]}


分词器还可以通过 `return_tensors="pt"` 来指定返回的张量类型，例如 `return_tensors="pt"` 会返回 PyTorch 张量，`return_tensors="tf"` 会返回 TensorFlow 张量，`return_tensors="np"` 会返回 NumPy 数组。例如：

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")  # padding=True 等同于 padding="longest"，按照批次（batch）中最长的序列进行填充
print(model_inputs)

model_inputs = tokenizer(sequences, padding=True, return_tensors="np")
print(model_inputs)

在实际使用分词器时，通常会将 `padding=True` 和 `truncation=True`，并设置返回的张量类型为 PyTorch 张量，这样就同一个 batch 中的序列都会 padding 到相同长度，同时超长序列也会被截断。

#### 编码句子对

除了对单端文本进行编码以外，对于 BERT 等包含“句子对”预训练任务的模型，它们的分词器都支持对句子对进行编码，例如：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
print(tokens)
# {'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 
#  'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 
#  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# ['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']

此时分词器会使用 `[SEP]` token 来拼接两个句子，输出形式为 `[CLS] sentence1 [SEP] sentence2 [SEP]` 的 token 序列，这也是 BERT 模型预期的“句子对”输入形式。

返回结果中除了前面提到的 `input_ids` 和 `attention_mask` 之外，还包含了 `token_type_ids`，它用来标记哪些 token 属于第一个句子，哪些 token 属于第二个句子。在这个例子中，`token_type_ids` 就是 `[0, 0, ..., 0, 1, 1, ..., 1]`。

可以看到，第一个句子的 `[CLS] sentence1 [SEP]` 部分对应的 `token_type_ids` 都是 0，第二个句子的 `sentence2 [SEP]` 部分对应的 `token_type_ids` 都是 1。

> 注意：如果选择其他模型，分词器的输出不一定会包含 `token_type_ids`，因此在使用时需要查看文档。

在实际使用中，我们通常不需要去关注分词器的输出是否包含 `token_type_ids`，只需要将分词器的输出传递给模型即可，分词器会根据 checkpoint 自动调整输出格式。

## 5.4 添加 Token

在实际操作中，我们可能经常遇到输入中需要包含特殊标记符的情况，例如使用 `[ENT_START]` 和 `[ENT_END]` 来标记实体的起始和结束位置。由于这些自定义 token 不在词表中，因此需要手动添加到分词器中。

例如直接使用 BERT 分词器处理下面的句子：

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sentence = 'Two [ENT_START] cars [ENT_END] collided in a [ENT_START] tunnel [ENT_END] this morning.'
print(tokenizer.tokenize(sentence))
# ['two', '[', 'en', '##t', '_', 'start', ']', 'cars', '[', 'en', '##t', '_', 'end', ']', 'collided', 'in', 'a', '[', 'en', '##t', '_', 'start', ']', 'tunnel', '[', 'en', '##t', '_', 'end', ']', 'this', 'morning', '.']

由于分词器无法识别 `[ENT_START]` 和 `[ENT_END]`，因此会将它们进行切分，例如 `[ENT_START]` 被切分成了 '[', 'en', '##t', '_', 'end', ']' 六个 token。

此外，一些领域的专业词汇，例如使用多个词语的缩写拼接而成的专有名词，同样也不在模型的词表中，因此也需要手动添加到分词器中。

### 添加新 Token

Transformers 库提供了两种方式来添加新 token，分别是：

1. `add_tokens()`: 添加普通 token，传入参数是需要添加的新 tokens 列表，如果 token 不在词表中，会直接添加到词表的末尾；
2. `add_special_tokens()`: 添加特殊 token，传入参数是包含特殊 token 的字典，键是 token 名称，键只能从 `bos_token`, `eos_token`, `unk_token`, `sep_token`, `pad_token`, `cls_token`, `mask_token`, `additional_special_tokens` 中选择，值是需要添加的 token。同样的，如果 token 不在词表中，会直接添加到词表的末尾，添加后还可以通过特殊属性来访问这些 token。

### 调整 embedding 矩阵

添加新 token 之后，需要调整模型的 embedding 矩阵，使得新 token 也有对应的 embedding。这可以通过调用 `resize_token_embeddings()` 方法来实现，例如：

In [ ]:
from transformers import AutoTokenizer, AutoModel

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)

print('vocabulary size:', len(tokenizer))
num_added_toks = tokenizer.add_tokens(['[ENT_START]', '[ENT_END]'], special_tokens=True)
print("After we add", num_added_toks, "tokens")
print('vocabulary size:', len(tokenizer))

model.resize_token_embeddings(len(tokenizer))
print(model.embeddings.word_embeddings.weight.size())

# Randomly generated matrix
print(model.embeddings.word_embeddings.weight[-2:, :])

在默认情况下，新添加 token 的 embedding 是随机初始化的。

## 5.5 Token Embedding 初始化

如果有充分的语料对模型进行微调或者继续预训练，那么将新添加的 token 初始化为随机向量没什么问题。但是如果训练语料有限，甚至是只有少量语料的 few-shot learning 场景下，这种做法就存在问题。研究表明，在训练数据不够多的情况下，这些新添加 token 的 embedding 只会在初始值附近小幅波动。换句话说，即使经过训练，他们的值事实上还是随机的。

### 直接赋值

例如将新添加的 token 都初始化为全零向量：

In [ ]:
import torch

with torch.no_grad():
    model.embeddings.word_embeddings.weight[-2:, :] = torch.zeros([2, model.config.hidden_size], requires_grad=True)
print(model.embeddings.word_embeddings.weight[-2:, :])

注意：初始化 embedding 的过程并不可导，因此这里通过 `torch.no_grad()` 来关闭梯度计算。

### 初始化为已有 token 的值

更高级的做法是根据新添加 token 的语义来进行初始化。例如将值初始化为 token 语义描述中所有 token 的平均值，假设新 token $t_i$ 的语义描述为 $w_{i,1}$, $w_{i,2}$, ..., $w_{i,n}$，那么新 token 的 embedding 可以初始化为：$E(t_i) = \frac{1}{n} \sum_{j=1}^{n} E(w_{i,j})$。

对于上面的例子，我们可以分别为 `[ENT_START]` 和 `[ENT_ENT]` 编写对应的语义描述，然后计算它们的平均值作为初始化值：

In [ ]:
descriptions = ['start of entity', 'end of entity']

with torch.no_grad():
    for i, token in enumerate(reversed(descriptions), start=1):
        tokenized = tokenizer.tokenize(token)
        print(tokenized)
        tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized)
        new_embedding = model.embeddings.word_embeddings.weight[tokenized_ids].mean(axis=0)
        model.embeddings.word_embeddings.weight[-i, :] = new_embedding.clone().detach().requires_grad_(True)
print(model.embeddings.word_embeddings.weight[-2:, :])